### Lineup Optimisation with Genetic Algorithms

In [22]:
import os
import glob
import random
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm

from constants import DATA_DIR
from utils import csv_concatenate, calculate_FPTS, calculate_MAE, calculate_RMSE, cross_val, load_full_dataset

In [23]:
import warnings
warnings.filterwarnings('ignore')

In [27]:
def add_positions(df):
    
    pos = {'PG':[], 'SG':[], 'SF':[], 'PF':[], 'C':[], 'G':[], 'F':[]}
    
    for i in range(df.shape[0]):
        for key in pos.keys():
            if key in df.loc[i, 'Pos']:
                pos[key].append(1)
            else:
                pos[key].append(0)
    
    for key in pos.keys():
        df[key] = pos[key]

### Genetic Algorithms

In [28]:
#return a list of lineups with indices in df_target
def create_random_lineups(df, num_lineups):
    lineups = {'PG':[], 'SG':[], 'SF':[], 'PF':[], 'C':[], 'G':[], 'F':[], 'Util':[]}
    n = df.shape[0]
    
    for i in range(num_lineups):
        for key in list(lineups.keys())[:7]:
            lineups[key].append(df[df[key]==1].sample(1).index[0])
        
        lineups['Util'].append(df.sample(1).index[0])
    
    df_lineups = pd.DataFrame(lineups).loc[:,['PG', 'SG', 'SF', 'PF', 'C', 'G', 'F', 'Util']]
    
    return df_lineups 

In [29]:
def calculate_fitness(df_lineups):
    fitness = []
    
    for i in range(df_lineups.shape[0]):
        
        salary = df_target.loc[df_lineups.loc[i,:].values, 'Salary'].sum()
        total_FPTS = df_target.loc[df_lineups.loc[i,:].values, 'Pred'].sum()
        
        #Check for duplicates
        if len(set(df_lineups.loc[i,:].values)) < 8:
            fitness.append(0)
        
        #Check for Salary Caps
        elif salary >= salary_cap:
            fitness.append(0)
            
        #Calculate the cumulative predicted FPTS
        else:
            fitness.append(total_FPTS)
        
    return fitness

In [30]:
def breed(df_lineups):  
    df_parents = df_lineups
    positions = list(df_lineups.columns)[:-1]
    df_children = pd.DataFrame([], columns=positions)
    
    df_fit = df_lineups.sort_values(by='Fitness', ascending=False).reset_index(drop=True)
    parents = df_fit.head(2)

    #Prevent breeding betwee two same lineups
    if all(parents.loc[0,:] != parents.loc[1,:]):
        next_index = 2

        while all(parents.loc[0,:] == parents.loc[1,:]):
            rows = [0, next_index]
            parents = df_fit.loc[rows]
            parents = parents.reset_index(drop=True)
            next_index += 1
            
    parents = parents.loc[:, positions]

    for pos in positions:
        #Add parents and swap each items position by position, creating 16 lineups
        df_children = df_children.append(parents, ignore_index=True)
        df_children.loc[df_children.shape[0]-2, pos] = parents.loc[1, pos]
        df_children.loc[df_children.shape[0]-1, pos] = parents.loc[0, pos]
    
    #Add parents for a total of 18 lineups
    df_children = df_children.append(parents).reset_index(drop=True)
    return df_children

In [31]:
def mutate(df_lineups, df_original, num_mutations):
    positions = list(df_lineups.columns)[:-1]
    df_mutants = pd.DataFrame([], columns=positions)
    df_parent = df_lineups.sort_values(by='Fitness', ascending=False).head(1).loc[:,positions]
    
    #Append the original
    df_mutants = df_mutants.append(df_parent)
    
    #Append the original and then mutate it by replacing it with a random sample for a random position
    for i in range(num_mutations):
        pos_to_swap = random.choice(positions)
        mutant_index = df_original.sample(1).index[0]
        
        df_mutants = df_mutants.append(df_parent, ignore_index=True)
        current_index = df_mutants.shape[0]-1
        df_mutants.loc[current_index, pos_to_swap] = df_original.loc[mutant_index, pos_to_swap]
        
    return df_mutants

In [32]:
def evolution(df_random_lineups, num_mutations, num_generations):
    df_init = df_random_lineups

    for i in range(num_generations):
    
        df_children = breed(df_init)
        df_children['Fitness'] = calculate_fitness(df_children)
        
        df_mutants = mutate(df_children, df_random_lineups, num_mutations)
        df_mutants['Fitness'] = calculate_fitness(df_mutants)
        
        df_init = df_mutants
    
    df_init['Fitness'] = calculate_fitness(df_init.drop('Fitness', axis=1))
    df_final = df_init.sort_values(by='Fitness', ascending=False).reset_index(drop=True)
    
    return df_final

In [42]:
def compounding_evolution(population_size=200, num_mutations=100, num_generations=50, num_compounding=5):
    optimal_lineups = []
    
    for i in tqdm(range(num_compounding)):
        df_random_lineups = create_random_lineups(df_target, population_size)
        df_random_lineups['Fitness'] = calculate_fitness(df_random_lineups)
        
        optimal = evolution(df_random_lineups, num_mutations, num_generations)
        
        optimal_lineups.append(list(optimal.loc[0,:].values[:-1].astype(int)))
        
    return optimal_lineups

In [43]:
np.random.seed(23)
salary_cap = 50000

df_pred = pd.read_csv(os.path.join(DATA_DIR, 'Predictions', '20190528-00h07m.csv'))

add_positions(df_pred)

In [44]:
df_cashline = pd.read_csv(os.path.join(DATA_DIR, 'Contests', 'cashline_march_2019.csv'))
df_cashline.head(5)

,Date,Type,Fee,Cashline
0,20190301,Double Up,50,335.50
1,20190302,Double Up,25,264.00
2,20190303,Double Up,25,282.25
3,20190304,Double Up,25,276.50
4,20190305,Double Up,25,280.25


In [45]:
date = []
pred = []
actual = []
cashline = []
win = []
earnings = []


for i in tqdm(range(df_cashline.shape[0])):
    df_target = df_pred.loc[(df_pred['Salary']!=0)&(df_pred['Date']==df_cashline.loc[i, 'Date'])].reset_index(drop=True)
    
    optimal_lineups = compounding_evolution()
    
    top_lineups = {'Lineup':[], 'Pred':[], 'Actual':[]}

    for j, indices in enumerate(optimal_lineups):
        top_lineups['Lineup'].append(j)
        top_lineups['Pred'].append(df_target.loc[indices, 'Pred'].sum())
        top_lineups['Actual'].append(df_target.loc[indices, 'FPTS'].sum())


    df_lineups = pd.DataFrame(top_lineups).sort_values(by='Pred', ascending=False).reset_index(drop=True)
    df_lineups = df_lineups.loc[:, ['Lineup', 'Pred', 'Actual']]

    rows = optimal_lineups[df_lineups.loc[0, 'Lineup']]
    df_best = df_target.loc[rows,:]
    display(df_best.loc[:,['Date', 'Name', 'Team', 'Pos', 'FPTS', 'Pred', 'Salary']])
    
    print('Salary:', df_target.loc[rows,'Salary'].sum())
    print('Predicted:', df_target.loc[rows,'Pred'].sum())
    print('Actual:', df_target.loc[rows,'FPTS'].sum())
    print('Cashline:', df_cashline.loc[i, 'Cashline'])
    
    pred.append(df_target.loc[rows,'Pred'].sum())
    actual.append(df_target.loc[rows,'FPTS'].sum())
    cashline.append(df_cashline.loc[i, 'Cashline'])
    
    if df_target.loc[rows, 'FPTS'].sum() >= df_cashline.loc[i, 'Cashline']:
        print('Win:', df_cashline.loc[i, 'Fee'])
        win.append(1)
        earnings.append(df_cashline.loc[i, 'Fee'])
    else:
        print('Lose:', df_cashline.loc[i, 'Fee'])
        win.append(0)
        earnings.append(-df_cashline.loc[i, 'Fee'])
    

,Date,Name,Team,Pos,FPTS,Pred,Salary
0,20190301,Trae Young,ATL,PG,87.00,44.212644,8100.0
57,20190301,Shai Gilgeous-Alexander,LAC,PG/SG,27.00,19.339563,3500.0
71,20190301,LeBron James,LAL,SF/PF,58.25,58.342864,11400.0
51,20190301,Marvin Williams,CHO,SF/PF,24.00,25.462783,4400.0
65,20190301,Ivica Zubac,LAC,C,31.00,23.394498,4000.0
3,20190301,Kevin Huerter,ATL,SG/SF,21.00,21.444520,3500.0
101,20190301,Anthony Davis,NOP,PF/C,40.00,39.174211,6800.0
45,20190301,Kemba Walker,CHO,PG,47.50,44.192811,8200.0


Salary: 49900.0
Predicted: 275.56389376442485
Actual: 335.75
Cashline: 335.5
Win: 50


,Date,Name,Team,Pos,FPTS,Pred,Salary
15,20190302,Jalen Brunson,DAL,PG/SG,12.00,26.142896,3800.0
118,20190302,Dennis Schroder,OKC,PG/SG,29.75,29.920274,5000.0
145,20190302,Tobias Harris,PHI,SF/PF,35.25,39.314941,7700.0
185,20190302,Derrick Favors,UTA,PF/C,54.00,26.499595,4800.0
33,20190302,Nikola Jokic,DEN,C,49.25,53.519790,10200.0
58,20190302,Rajon Rondo,LAL,PG,18.00,31.046113,5300.0
90,20190302,Kelly Olynyk,MIA,PF/C,45.00,30.119090,5100.0
114,20190302,Julius Randle,NOP,PF/C,52.50,40.988069,8000.0


Salary: 49900.0
Predicted: 277.55076833487703
Actual: 295.75
Cashline: 264.0
Win: 25


,Date,Name,Team,Pos,FPTS,Pred,Salary
108,20190303,Dennis Schroder,OKC,PG/SG,47.50,32.405513,5400.0
46,20190303,Wayne Ellington,DET,SG/SF,22.75,21.220579,3700.0
123,20190303,Aaron Gordon,ORL,SF/PF,31.00,33.906428,6700.0
39,20190303,Larry Nance,CLE,PF/C,27.25,33.963286,5800.0
49,20190303,Blake Griffin,DET,PF/C,37.75,44.910241,8400.0
134,20190303,Jeremy Lin,TOR,PG/SG,6.75,23.690695,4100.0
3,20190303,Kent Bazemore,ATL,SG/SF,23.25,26.408299,4600.0
53,20190303,James Harden,HOU,PG/SG,63.75,64.016959,11200.0


...

Salary: 49800.0
Predicted: 276.26888671860274
Actual: 270.75
Cashline: 247.5
Win: 25


,Date,Name,Team,Pos,FPTS,Pred,Salary
194,20190330,Kyle Lowry,TOR,PG,37.75,32.884318,6800.0
13,20190330,D'Angelo Russell,BRK,PG/SG,53.25,45.258991,8200.0
177,20190330,Evan Turner,POR,SG/SF,8.00,21.402691,3300.0
189,20190330,Marvin Bagley,SAC,PF/C,37.50,38.061072,7100.0
134,20190330,Mitchell Robinson,NYK,C,40.00,30.728935,4900.0
126,20190330,Emmanuel Mudiay,NYK,PG,35.25,34.865762,5500.0
82,20190330,JaMychal Green,LAC,SF/PF,37.00,26.456967,3800.0
170,20190330,Damian Lillard,POR,PG,41.50,46.600031,10100.0


Salary: 49700.0
Predicted: 276.2587666619512
Actual: 290.25
Cashline: 293.25
Lose: 25


,Date,Name,Team,Pos,FPTS,Pred,Salary
116,20190331,Derrick White,SAS,PG/SG,14.75,26.812919,4900.0
71,20190331,Delon Wright,MEM,PG/SG,44.00,27.688735,5500.0
56,20190331,Landry Shamet,LAC,SG/SF,13.75,18.806117,3900.0
95,20190331,Paul George,OKC,SF/PF,52.25,51.186916,9600.0
45,20190331,Draymond Green,GSW,PF/C,31.25,33.856728,5900.0
91,20190331,Russell Westbrook,OKC,PG,63.75,62.368886,10700.0
69,20190331,Moritz Wagner,LAL,PF/C,20.25,17.813266,3300.0
49,20190331,Shai Gilgeous-Alexander,LAC,PG/SG,26.50,32.606067,6000.0


Salary: 49800.0
Predicted: 271.13963329055423
Actual: 266.5
Cashline: 240.5
Win: 25


In [48]:
print(sum(win)/len(win))
print(sum(earnings))

0.3548387096774194
-215


In [ ]:
date = list(set(df_cashline['Date'].values))
df_result = pd.DataFrame({'Date': date,
                          'Predicted':pred,
                          'Actual': actual,
                          'Cashline': cashline,
                          'Win':win,
                          'Earnings': earnings
                         })

df_result.loc[:,['Date', 'Predicted', 'Actual', 'Cashline', 'Win', 'Earnings']]

### Baseline - random 10,000

In [ ]:
df_result.loc[:,['Date', 'Predicted', 'Actual', 'Cashline', 'Win', 'Earnings']]

In [ ]:
df_random_lineups = create_random_lineups(df_target, 200)
df_random_lineups['Fitness'] = calculate_fitness(df_random_lineups)
df_random_lineups.sort_values(by='Fitness', ascending=False).head(10)

### Visualisation

In [ ]:
df_baseline = csv_concatenate(os.path.join(data_dir, data_dir, 'Dataframes', 'modelling', 'baseline'))
df_baseline['Baseline'] = calculate_FPTS(df_baseline)

In [ ]:
#target_date = 20180310
target_date = 20180326

In [ ]:
df_target = df_pred.loc[(df_pred['Salary']!=0)&(df_pred['Date']==target_date)].reset_index(drop=True)
    
optimal_lineups = compounding_evolution(population_size=200, num_mutations=50,
                                        num_generations=100, num_compounding=5)

top_lineups = {'Lineup':[], 'Pred':[], 'Actual':[]}

for j, indices in enumerate(optimal_lineups):
    top_lineups['Lineup'].append(j)
    top_lineups['Pred'].append(df_target.loc[indices, 'Pred'].sum())
    top_lineups['Actual'].append(df_target.loc[indices, 'FPTS'].sum())


df_lineups = pd.DataFrame(top_lineups).sort_values(by='Pred', ascending=False).reset_index(drop=True)
df_lineups = df_lineups.loc[:, ['Lineup', 'Pred', 'Actual']]

rows = optimal_lineups[df_lineups.loc[0, 'Lineup']]
df_best = df_target.loc[rows,:]
display(df_best.loc[:,['Date', 'Name', 'Team', 'Pos', 'FPTS', 'Pred', 'Salary']])
print('Salary:', df_target.loc[rows,'Salary'].sum())
print('Predicted:', df_target.loc[rows,'Pred'].sum())
print('Actual:', df_target.loc[rows,'FPTS'].sum())
print('Cashline:', int(df_cashline.loc[df_cashline['Date']==target_date, 'Cashline'].values))

In [ ]:
print(calculate_MAE(df_best['FPTS'].values, df_best['Pred'].values))
print(calculate_RMSE(df_best['FPTS'], df_best['Pred']))

In [ ]:
df_best

In [ ]:
Baseline = []

for i in range(df_best.shape[0]):
    df_best = df_best.reset_index(drop=True)
    Baseline.append(df_baseline.loc[(df_baseline['Name']==df_best.loc[i, 'Name'])&(df_baseline['Date']==df_best.loc[i,'Date']), 'Baseline'])

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

names = [name.split(' ')[0][:1] + '. '+ name.split(' ')[1] for name in df_best['Name']]
positions = ['(PG)', '(SG)', '(SF)','(PF)','(C)','(G)','(F)','(Utility)']

names = [names[i]+'\n'+positions[i] for i in range(8)]

trace1 = go.Bar(
    x=names,
    y=df_best['FPTS'],
    name='Actual FPTS'
)

trace2 = go.Bar(
    x=names,
    y=df_best['Pred'],
    name='Prediction'
)

trace3 = go.Bar(
    x=names,
    y=Baseline,
    name='Baseline'
)


data = [trace1, trace2, trace3]
layout = go.Layout(
    
)

layout = go.Layout(
        title = 'FPTS: Actual and Predicted',
        barmode='group',
        legend = {"x":0.85, 'y':0.95, 'borderwidth': 1},
        yaxis = {"title":"FPTS"},
    )

config={'showLink': False}

fig = go.Figure(data=data, layout=layout)
plot_url = py.plot(fig, filename='prediction0326')
py.iplot(fig, filename='prediction0326')